In [ ]:
from load_ulg import load_ulg
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model_name = "fs"
if model_name == "fs":
    ulog_files = [
        "logs/2024-01-08-pavel-fs/log_63_2024-1-8-16-37-54.ulg",
        "logs/2024-01-08-pavel-fs/log_64_2024-1-8-16-39-44.ulg",
        "logs/2024-01-08-pavel-fs/log_65_2024-1-8-16-40-52.ulg",
        "logs/2024-01-08-pavel-fs/log_66_2024-1-8-16-42-48.ulg",
        "logs/2024-01-12-pavel policy test/log_76_2024-1-12-17-25-30.ulg",
        "logs/2024-01-12-pavel policy test/log_77_2024-1-12-17-27-50.ulg"
    ]
elif model_name == "x500":
    ulog_files = [
        "logs/2023-12-27-x500/log_36_2023-12-27-14-21-32.ulg",
        "logs/2023-12-27-x500/log_37_2023-12-27-14-24-24.ulg"
    ]
elif model_name == "race6":
    ulog_files = [
        "logs/2023-12-13-nishanth-sysid/log_45_2023-12-13-21-07-46-up-and-down.ulg",
        "logs/2023-12-14-yang-crash/log_60_2023-12-14-20-26-16.ulg",
        "logs/2023-12-13-nishanth-sysid/log_46_2023-12-13-21-08-08-left-and-right.ulg"
    ]

dfs = [load_ulg(file) for file in ulog_files]

In [ ]:
output_topic = "actuator_motors_mux"
inertia_ratio = 1.879
exponents_thrust_curve = [0, 2]

In [ ]:

# we want to do something
# we have selected flights to use for this
# cut out the relevant timeframe
# we slice the flights based on gaps and interpolate
# we then preprocess the data with time correlation (depends on the motor model) => makes the data iid
# we use the iid data to estimate parameters

if model_name == "fs":
    rotor_x_displacement = 0.4179/2
    rotor_y_displacement = 0.481332/2
    # model is in FLU frame
    model = {
        "gravity": 9.81,
        "mass": 2.3 + 1.05,
        "rotor_positions": np.array([
            [ rotor_x_displacement, -rotor_y_displacement, 0],
            [-rotor_x_displacement,  rotor_y_displacement, 0],
            [ rotor_x_displacement,  rotor_y_displacement, 0],
            [-rotor_x_displacement, -rotor_y_displacement, 0]
        ]),
        "rotor_thrust_directions": np.array([
            [0, 0, 1],
            [0, 0, 1],
            [0, 0, 1],
            [0, 0, 1]
        ]),
        "rotor_torque_directions": np.array([
            [0, 0, -1],
            [0, 0, -1],
            [0, 0,  1],
            [0, 0,  1]
        ])
    }
    g = np.array([0, 0, -model["gravity"]])
elif model_name == "x500":
    rotor_x_displacement = 0.1765
    rotor_y_displacement = 0.1765
    model = {
        "gravity": 9.81,
        "mass": 2,
    
        "rotor_positions": np.array([
            [ rotor_x_displacement, -rotor_y_displacement, 0],
            [-rotor_x_displacement,  rotor_y_displacement, 0],
            [ rotor_x_displacement,  rotor_y_displacement, 0],
            [-rotor_x_displacement, -rotor_y_displacement, 0]
        ]),
        "rotor_thrust_directions": np.array([
            [0, 0, 1],
            [0, 0, 1],
            [0, 0, 1],
            [0, 0, 1]
        ]),
        "rotor_torque_directions": np.array([
            [0, 0, -1],
            [0, 0, -1],
            [0, 0,  1],
            [0, 0,  1]
        ])
    }
    g = np.array([0, 0, -model["gravity"]])

# Step 0: Load the data

In [ ]:
wanted_columns = [
    *[f"vehicle_attitude_q[{i}]" for i in range(4)],
    *[f"vehicle_acceleration_xyz[{i}]" for i in range(3)],
    *[f"vehicle_angular_velocity_xyz[{i}]" for i in range(3)],
    *[f"vehicle_angular_velocity_xyz_derivative[{i}]" for i in range(3)],
    *[f"{output_topic}_control[{i}]" for i in range(4)],
]

# flights => timeseries (timestamp + single value) => split up into separate time-series based on gaps

flights = [
    {
        "name": name,
        "data": {
            name: {
                "timestamps": np.array(c.index),
                "values": np.array(c)
            } for name, c in [(wc, df[wc].dropna()) for wc in wanted_columns]
        }
    } for name, df in zip(ulog_files, dfs)
]

### API
List of separate flights (probably from the same .ulog, arm->disarm). 
```
flights = {
    "name": "flight1.ulg",
    "data": {
        "vehicle_attitude_q[0]": {
            "timestamps": [0, 0.001, 0.002, ...],
            "values": [1, 0.99, 0.98, ...],
        },
        "vehicle_attitude_q[1]": {...},
        ...
        "vehicle_acceleration_xyz[0]": {...},
        ...
        "vehicle_angular_velocity_xyz[0]": {...},
        ...
        "vehicle_angular_velocity_xyz_derivative[0]": {...},
        ...
        "actuator_motors_control[0]": {...},
        ...
    }
}
```
Note: The timestamps do not need to be synchronized, and the frequencies of logging can vary between the timeseries. They will be synchronized and interpolated based on the highest frequency one

# Step 1: Plot the flights

In [ ]:

for flight_i, flight in enumerate(flights):
    # Assuming you have 4 motor plots and 4 additional plots for thrust and torque analyses
    fig, axs = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
    fig.suptitle(f"Flight {flight_i}: {flight['name']}")

    # Plot motor controls
    ax = 0
    for motor_i, motor in enumerate([f"{output_topic}_control[{i}]" for i in range(4)]):
        axs[ax].plot(flight["data"][motor]["timestamps"], flight["data"][motor]["values"], label=f"motor {motor_i}")
        axs[ax].legend(loc='upper left', bbox_to_anchor=(1,1))
        axs[ax].set_ylabel('Motor Control')
    ax += 1
    # Calculate thrust, thrust torque, and geometric torque values
    thrust_values = []
    thrust_torque_values = []
    geometric_torque_values = []
    timestamps = None
    for motor_i, motor in enumerate([f"{output_topic}_control[{i}]" for i in range(4)]):
        timestamps = flight["data"][motor]["timestamps"]
        thrust_values.append(flight["data"][motor]["values"][:, np.newaxis] * model["rotor_thrust_directions"][motor_i][np.newaxis, :])
        thrust_torque_values.append(flight["data"][motor]["values"][:, np.newaxis] * model["rotor_torque_directions"][motor_i][np.newaxis, :])
        geometric_torque_values.append(flight["data"][motor]["values"][:, np.newaxis] * np.cross(model["rotor_positions"][motor_i], model["rotor_thrust_directions"][motor_i])[np.newaxis, :])
    thrust_values = sum(thrust_values)
    thrust_torque_values = sum(thrust_torque_values)
    geometric_torque_values = sum(geometric_torque_values)

    axs[ax].set_title("thrust (z)")
    axs[ax].plot(timestamps, thrust_values[:, 2], label="z")
    axs[ax].legend(loc='upper left', bbox_to_anchor=(1,1))
    axs[ax].set_ylabel('Thrust (z)')
    ax += 1
    
    axs[ax].set_title("thrust -> torque")
    axs[ax].plot(timestamps, thrust_torque_values[:, 2], label="z")
    axs[ax].legend(loc='upper left', bbox_to_anchor=(1,1))
    axs[ax].set_ylabel('Torque (z)')
    ax += 1

    axs[ax].set_title("thrust -> geometry -> torque")
    axs[ax].plot(timestamps, geometric_torque_values[:, 0], label="x")
    axs[ax].legend(loc='upper left', bbox_to_anchor=(1,1))
    axs[ax].set_ylabel('Geometry Torque (x)')
    axs[ax].plot(timestamps, geometric_torque_values[:, 1], label="y")
    axs[ax].legend(loc='upper left', bbox_to_anchor=(1,1))
    axs[ax].set_ylabel('Geometry Torque (y)')
    ax += 1

    axs[-1].set_xlabel('Timestamps')
    for ax in axs[:-1]:  # Loop through all axes except the bottommost
        ax.xaxis.set_tick_params(labelbottom=True)

    plt.tight_layout(rect=[0, 0, 1, 0.99])  # Adjust the layout to make room for the suptitle
    plt.show()

# Step 2: Select Flights and Time Slices

In [ ]:
if model_name == "fs":
    timeframes_thrust = [
        {
            "flight": 0,
            "start": 10,
            "end": 45
        }
    ]
    timeframes_inertia_roll_pitch = [
        {
            "flight": 1,
            "start": 10,
            "end": 17
        },
        {
            "flight": 1,
            "start": 19,
            "end": 35
        },
        {
            "flight": 2,
            "start": 10,
            "end": 45
        }
    ]
    timeframes_inertia_yaw = [
        {
            "flight": 3,
            "start": 15,
            "end": 30
        }
    ]
elif model_name == "x500":
    timeframes_thrust = [
        {
            "flight": 0,
            "start": 30,
            "end": 45
        }
    ]
    timeframes_inertia_roll_pitch = [
        {
            "flight": 1,
            "start": 10,
            "end": 40
        }
    ]
    timeframes_inertia_yaw = [
        {
            "flight": 1,
            "start": 10,
            "end": 40
        }
    ]

In [ ]:
def extract_timeframes(flights, timeframes):
    output_flights = []
    fragment_counter = {}
    for timeframe in timeframes:
        flight = deepcopy(flights[timeframe["flight"]])
        fragment_id = 0 if flight["name"] not in fragment_counter else fragment_counter[flight["name"]]
        fragment_counter[flight["name"]] = fragment_id + 1
        flight["name"] = flight["name"] + f".{fragment_id}"
        start, end = timeframe["start"], timeframe["end"]
        data = flight["data"]
        for series in data:
            mask = (data[series]["timestamps"] > start) & (data[series]["timestamps"] < end)
            data[series]["timestamps"] = data[series]["timestamps"][mask]
            data[series]["values"] = data[series]["values"][mask]
        output_flights.append(flight)
    return output_flights

In [ ]:
flights_thrust = extract_timeframes(flights, timeframes_thrust)
flights_inertia_roll_pitch = extract_timeframes(flights, timeframes_inertia_roll_pitch)
flights_inertia_yaw = extract_timeframes(flights, timeframes_inertia_yaw)

In [ ]:
[f["name"] for f in flights_inertia_roll_pitch]

# Step 3: Detect gaps and interpolate data

In [ ]:
def slice_gaps_and_interpolate(flights):
    flights_output = []
    for flight in flights:
        lowest_frequency = None
        lowest_frequency_name = None
        highest_frequency = None
        highest_frequency_name = None
        for name, data in flight["data"].items():
            diff = np.diff(data["timestamps"])
            frequency = 1/np.median(diff)
            if lowest_frequency is None or frequency < lowest_frequency:
                lowest_frequency = frequency
                lowest_frequency_name = name
            if highest_frequency is None or frequency > highest_frequency:
                highest_frequency = frequency
                highest_frequency_name = name
            # print(f"{name}: {1/np.median(diff)}")

        interval_threshold = 3 * 1/lowest_frequency
        print(f"Lowest frequency: {lowest_frequency} for {lowest_frequency_name}")
        print(f"Highest frequency: {highest_frequency} for {highest_frequency_name}")

        earliest_timestamp_all = max([data["timestamps"][0] for name, data in flight["data"].items()])
        latest_timestamp_all = min([data["timestamps"][-1] for name, data in flight["data"].items()])
        print(f"Earliest timestamp_all: {earliest_timestamp_all}")
        print(f"Latest timestamp_all: {latest_timestamp_all}")
        master_timestamps_full = flight["data"][highest_frequency_name]["timestamps"]
        master_timestamps = master_timestamps_full[(master_timestamps_full > earliest_timestamp_all) & (master_timestamps_full < latest_timestamp_all)]
        earliest_timestamp = master_timestamps[0]
        latest_timestamp = master_timestamps[-1]
        print(f"Cutting {(100 * (1 - len(master_timestamps) / len(master_timestamps_full))):.2f}% of the data to synchronize the timestamp start and end")

        total_time = latest_timestamp - earliest_timestamp

        gaps = []
        for name, data in flight["data"].items():
            current_timestamps_full = data["timestamps"]
            current_timestamps = current_timestamps_full[(current_timestamps_full > earliest_timestamp) & (current_timestamps_full < latest_timestamp)]
            current_timestamps_augmented = np.concatenate([[earliest_timestamp], current_timestamps, [latest_timestamp]])
            diff = np.diff(current_timestamps_augmented)
            current_gaps = np.where(diff > interval_threshold)[0]
            for gap in current_gaps:
                gap_start = data["timestamps"][gap]
                gap_end = data["timestamps"][gap+1]
                gaps.append((gap_start, gap_end))
        gaps_sorted = sorted(gaps, key=lambda x: x[0])


        current_gap_start = None
        current_gap_end = None
        combined_gaps = []

        for i, (gap_start, gap_end) in enumerate(gaps_sorted):
            if current_gap_start is None:
                current_gap_start = gap_start
            
            if current_gap_end is None:
                current_gap_end = gap_end
            
            if gap_end > current_gap_end:
                current_gap_end = gap_end
            
            if i < len(gaps_sorted) - 1:
                next_gap_start, next_gap_end = gaps_sorted[i+1]
                if next_gap_start - current_gap_end > interval_threshold:
                    print(f"Gap: {current_gap_start} - {current_gap_end}")
                    combined_gaps.append((current_gap_start, current_gap_end))
                    current_gap_start = None
                    current_gap_end = None
            else:
                print(f"Gap start {gap_start} - {gap_end}")
                print(f"Final Gap: {current_gap_start} - {current_gap_end}")
                combined_gaps.append((current_gap_start, current_gap_end))
        print(f"Number of gaps: {len(combined_gaps)}")

        total_gap_time = sum([gap_end - gap_start for gap_start, gap_end in combined_gaps])
        assert total_gap_time < 0.1 * total_time, f"Total gap time: {total_gap_time:.2f}s"

        subflights = []
        current_segment_start_timestamp = earliest_timestamp
        for gap_start, gap_end in [*combined_gaps, (latest_timestamp, latest_timestamp)]:
            segment_time = gap_start - current_segment_start_timestamp
            if segment_time > 0.01 * total_time:
                current_segment_timestamps = master_timestamps[(master_timestamps > current_segment_start_timestamp) & (master_timestamps < gap_start)]
                sub_flight = {
                    name: {
                        "timestamps": current_segment_timestamps,
                        "values": np.interp(current_segment_timestamps, data["timestamps"], data["values"])
                    } for name, data in flight["data"].items()
                }
                subflights.append(sub_flight)
            else:
                print(f"Skipping segment of length {segment_time:.2f}s")
            current_segment_start_timestamp = gap_end
        print(f"Number of subflights: {len(subflights)}")


        for subflight in subflights:
            plt.figure()
            plt.title(flight["name"])
            for i, key in enumerate([f"actuator_motors_mux_control[{i}]" for i in range(4)]):
                ts = subflight[key]
                plt.plot(ts["timestamps"], ts["values"], label=f"motor {i}")
            plt.legend()
        
        for subflight_i, subflight in enumerate(subflights):
            flights_output.append({
                "name": flight["name"] + f"_{subflight_i}",
                "timestamps": subflight[highest_frequency_name]["timestamps"],
                "data": subflight
            })
    return flights_output

In [ ]:
sliced_and_interpolated_flights_thrust = slice_gaps_and_interpolate(flights_thrust)
sliced_and_interpolated_flights_inertia_roll_pitch = slice_gaps_and_interpolate(flights_inertia_roll_pitch)
sliced_and_interpolated_flights_inertia_yaw = slice_gaps_and_interpolate(flights_inertia_yaw)

# Step 4: Apply stateful transformation (filter motor data)

In [ ]:

def filter_ema(timestamps, rpm_setpoints, T_m):
    rpms_filtered = []
    rpm = None
    previous_t = None
    for t, rpm_setpoint in zip(timestamps, rpm_setpoints):
        if rpm is None:
            rpm = rpm_setpoint
        else:
            delta_t = t - previous_t
            alpha = np.exp(-delta_t / T_m)
            rpm = alpha*rpm + (1-alpha) * rpm_setpoint
        rpms_filtered.append(rpm)
        previous_t = t
    return np.array(rpms_filtered)

def combine(flights, T_m, frd=True, T_omega=0.05, thrust_curves=None):
    assert thrust_curves is None or len(thrust_curves) == 4
    from scipy.spatial.transform import Rotation as R
    def FRD2FLU(x):
        return np.array([x[0], -x[1], -x[2]])

    def FRD2FLU_quat(q):
        return R.from_quat([q[1], -q[2], -q[3], q[0]])
    from copy import deepcopy
    flights = deepcopy(flights)
    for flight in flights:
        timestamps = flight["timestamps"]
        # num_steps = len(timestamps)
        rpm_setpoints = np.array([flight["data"][f"{output_topic}_control[{i}]"]["values"] for i in range(4)]).T
        flight["rpm_setpoints"] = rpm_setpoints
        flight["rpms"] = filter_ema(timestamps, rpm_setpoints, T_m)
        acceleration = np.array([flight["data"][f"vehicle_acceleration_xyz[{i}]"]["values"] for i in range(3)]).T
        flight["acceleration"] = np.array(list(map(FRD2FLU, acceleration))) if frd else acceleration
        omega_original_frame = np.array([flight["data"][f"vehicle_angular_velocity_xyz[{i}]"]["values"] for i in range(3)]).T
        omega = np.array(list(map(FRD2FLU, omega_original_frame))) if frd else omega_original_frame
        flight["omega"] = omega
        domega_original_frame = np.array([flight["data"][f"vehicle_angular_velocity_xyz_derivative[{i}]"]["values"] for i in range(3)]).T
        domega = np.array(list(map(FRD2FLU, domega_original_frame))) if frd else domega_original_frame
        flight["domega"] = domega
        flight["domega_smooth"] = filter_ema(timestamps, domega, T_omega)
        omega_fd = (omega[1:] - omega[:-1]) / (timestamps[1:] - timestamps[:-1]).reshape(-1, 1)
        flight["domega_fd"] = np.concatenate([omega_fd, [omega_fd[-1]]])
        flight["domega_gradient"] = np.gradient(omega, timestamps, axis=0)
        if thrust_curves is not None:
            def apply_thrust_curves(rpms, thrust_curves):
                thrusts = []
                for thrust_curve_motor, rpm in zip(thrust_curves, rpms):
                    acc = 0
                    for exponent, coeff in zip(range(3), thrust_curve_motor):
                        acc += rpm**exponent * coeff
                    thrusts.append(acc)
                return np.array(thrusts)
            flight["thrusts"] = np.array([apply_thrust_curves(rpms, thrust_curves) for rpms in flight["rpms"]])
            motor_thrust_to_torque_geometric = np.array([np.cross(model["rotor_positions"][motor_i], model["rotor_thrust_directions"][motor_i]) for motor_i in range(4)])
            flight["pre_torque"] = flight["thrusts"][:, :, np.newaxis] * model["rotor_torque_directions"]
            flight["pre_torque_geometric"] = flight["thrusts"][:, :, np.newaxis] * motor_thrust_to_torque_geometric
        
    
    thrusts_combined = {}
    if thrust_curves is not None:
        thrusts_combined = {
            "thrusts": np.concatenate([flight["thrusts"] for flight in flights]),
            "pre_torque": np.concatenate([flight["pre_torque"] for flight in flights]),
            "pre_torque_geometric": np.concatenate([flight["pre_torque_geometric"] for flight in flights]),
        }

    return {
        "rpm_setpoints": np.concatenate([flight["rpm_setpoints"] for flight in flights]),
        "rpms": np.concatenate([flight["rpms"] for flight in flights]),
        "acceleration": np.concatenate([flight["acceleration"] for flight in flights]),
        "omega": np.concatenate([flight["omega"] for flight in flights]),
        "domega": np.concatenate([flight["domega"] for flight in flights]),
        "domega_smooth": np.concatenate([flight["domega_smooth"] for flight in flights]),
        "domega_fd": np.concatenate([flight["domega_fd"] for flight in flights]),
        "domega_gradient": np.concatenate([flight["domega_gradient"] for flight in flights]),
        **thrusts_combined
    }, flights


In [ ]:
T_m_test = 0.05
combined_test, _ = combine(sliced_and_interpolated_flights_thrust, T_m_test)

In [ ]:
def estimate_motor_parameters(combined, model, exponents, verbose=True, separate=True):
    b = []
    A = []


    num_steps = len(combined["rpms"])
    for step_i in range(num_steps):
        acceleration = combined["acceleration"][step_i]
        b.append(model["mass"] * acceleration)
        current_A = []
        rpm = combined["rpms"][step_i]
        if separate:
            for motor_i in range(4):
                for exponent in exponents:
                    current_A.append(model["rotor_thrust_directions"][motor_i] * rpm[motor_i] ** exponent)
        else:
            for exponent in exponents:
                acc = np.zeros(3)
                for motor_i in range(4):
                    acc += model["rotor_thrust_directions"][motor_i] * rpm[motor_i] ** exponent
                current_A.append(acc)
        A.append(np.array(current_A).T)

    A = np.array(A).reshape(-1, (4 if separate else 1) * len(exponents))
    b = np.array(b).reshape(-1)
    if verbose:
        print(f"A shape: {A.shape}")
        print(f"b shape: {b.shape}")
    K_f = np.linalg.lstsq(A, b, rcond=None)[0].reshape(-1, len(exponents))
    rmse = np.sqrt(np.mean((A @ K_f.reshape(-1) - b) ** 2))
    if separate:
        K_f_full = np.array([[K_f[motor_i][exponents.index(i)] if i in exponents else 0 for i in range(3)] for motor_i in range(4)])
    else:
        K_f_full = np.array([[K_f[0][exponents.index(i)] if i in exponents else 0 for i in range(3)] for motor_i in range(4)])
    return K_f_full, rmse

In [ ]:

K_f_test, rmse = estimate_motor_parameters(combined_test, model, exponents_thrust_curve)
print(f"K_f_test: {K_f_test}")
K_f_test_mean = K_f_test.mean(axis=0)
print(f"K_f_test_mean: {K_f_test_mean}")
# rmse = motor_model_rmse(K_f_mean, combined, model, exponents)

In [ ]:
from tqdm import tqdm
from tqdm.notebook import tqdm

T_m_candidates = np.linspace(0.001, 0.3, 20)
results = [(T_m, *estimate_motor_parameters(combine(sliced_and_interpolated_flights_thrust, T_m)[0], model, exponents_thrust_curve, verbose=False, separate=False)) for T_m in tqdm(T_m_candidates)]
T_m, K_fs, rmses = zip(*results)

In [ ]:
T_m = T_m_candidates[np.argmin(rmses)]
K_f = K_fs[np.argmin(rmses)]
K_f_mean = K_f.mean(axis=0)
print(f"K_f: {K_f}")
print(f"K_f_mean: {K_f_mean}")
plt.figure()
plt.plot(T_m_candidates, rmses, label=f"T_m = {T_m}")
plt.legend()
plt.show()

thrust_curves = np.array([K_f_mean for _ in range(4)])
# thrust_curves = K_f

combined_thrust, flights_thrust_processed = combine(sliced_and_interpolated_flights_thrust, T_m, thrust_curves=thrust_curves)
combined_inertia_roll_pitch, flights_inertia_roll_pitch_processed = combine(sliced_and_interpolated_flights_inertia_roll_pitch, T_m, thrust_curves=thrust_curves)
combined_inertia_yaw, flights_inertia_yaw_processed = combine(sliced_and_interpolated_flights_inertia_yaw, T_m, thrust_curves=thrust_curves)

In [ ]:
plt.scatter(combined_thrust["thrusts"].sum(axis=1), combined_thrust["acceleration"][:, 2]*model["mass"], s=0.05) 
plt.plot([0, 100], [0, 100], color="red")
K_f_mean

In [ ]:
I = []
for axis_i, axis_name in zip([0, 1], ["x", "y"]):
    dw_full = combined_inertia_roll_pitch["domega"][:, axis_i]
    torque_full = combined_inertia_roll_pitch["pre_torque_geometric"].sum(axis=1)[:, axis_i]
    perc = 50
    dw_perc_lower = np.percentile(dw_full, perc)
    dw_perc_upper = np.percentile(dw_full, 100-perc)
    print(f"domega lower: {dw_perc_lower:.3f} upper: {dw_perc_upper:.3f}")
    mask = (dw_full < dw_perc_lower) | (dw_full > dw_perc_upper)
    torques_perc_lower = np.percentile(torque_full, perc)
    torques_perc_upper = np.percentile(torque_full, 100-perc)
    # mask &= (torque_full < torques_perc_lower) | (torque_full > torques_perc_upper)
    print(f"masking rate: {1-mask.mean():.3f}")
    dw = dw_full[mask]
    torque = torque_full[mask]

    I_axis = 1/(dw**2).sum() * np.inner(dw, torque)
    I_inv_axis = 1/(torque**2).sum() * np.inner(dw, torque)
#     I_axis = 1/I_inv_axis
    print(f"I_{axis_name}{axis_name}: {I_axis:.3f}")
    plt.figure()
    plt.title(f"Axis: {axis_name}")
    if perc < 50:
        plt.scatter(torque_full, dw_full, s=0.1)
    plt.scatter(torque, dw, s=0.1)
    torques_in = np.linspace(torque_full.min(), torque_full.max(), 100)
    plt.plot(torques_in, torques_in / (I_axis), color="red")
    plt.plot(torques_in, torques_in / (1/I_inv_axis), color="red")
    plt.savefig(f"figures/inertia_{axis_name}.pdf")
    plt.show()
    I.append(I_axis)

In [ ]:
I_xx, I_yy = I[:2]
I_zz = (I_xx + I_yy)/2 * inertia_ratio
I = [I_xx, I_yy, I_zz]
print(f"I_xx: {I_xx:.4f}, I_yy: {I_yy:.4f}, I_zz: {I_zz:.4f},")

In [ ]:
K_ds = []
perc_candidates = [50] #np.linspace(0.1, 50, 20)
for perc in perc_candidates:
    combined_inertia_yaw["thrusts"].shape
    model["rotor_torque_directions"].shape
    thrust_torques_inputs = combined_inertia_yaw["thrusts"][:, :, np.newaxis] * model["rotor_torque_directions"]
    thrust_torque_inputs_z_full = thrust_torques_inputs[:, :, 2].sum(axis=1)
    dwz_full = combined_inertia_yaw["domega"][:, 2]

    thrust_torque_input_perc_lower = np.percentile(thrust_torque_inputs_z_full, perc)
    thrust_torque_input_perc_upper = np.percentile(thrust_torque_inputs_z_full, 100-perc)
    mask = (thrust_torque_inputs_z_full < thrust_torque_input_perc_lower) | (thrust_torque_inputs_z_full > thrust_torque_input_perc_upper)

    dwz_perc_lower = np.percentile(dwz_full, perc)
    dwz_perc_upper = np.percentile(dwz_full, 100-perc)
    mask &= (dwz_full < dwz_perc_lower) | (dwz_full > dwz_perc_upper)

    thrust_torque_inputs_z = thrust_torque_inputs_z_full[mask]
    dwz = dwz_full[mask]

    b_full = dwz_full * I_zz
    b = dwz * I_zz

    K_d = 1/(thrust_torque_inputs_z ** 2).sum() * np.inner(thrust_torque_inputs_z, b)
    print(f"b: {(b**2).sum()}")
    K_ds.append(K_d)

    plt.figure()
    plt.scatter(thrust_torque_inputs_z_full, b_full, s=0.01)
    # plt.scatter(thrust_torque_inputs_z, dwz, s=0.1, color="red")
    plt.plot(thrust_torque_inputs_z, K_d * thrust_torque_inputs_z, color="red", label=f"$K_d$ = {K_d:.3f}")
    plt.ylabel(f"Angular Acceleration (z) $\\times$ $I_{{zz}}$ ")
    plt.xlabel(f"Normalized Torque (z)")
    plt.legend()
    plt.savefig(f"figures/K_d_{perc}.pdf")
    plt.show()


    print(f"K_d: {K_d:.3f}")

In [ ]:
plt.scatter(combined_thrust["thrusts"].sum(axis=1), combined_thrust["acceleration"][:, 2]*model["mass"])
plt.plot([0, 100], [0, 100], color="red")